In [7]:
import os
import pandas as pd
from langchain.llms import OpenAI
from langchain.agents import (
    load_tools,
    initialize_agent,
    AgentType,
)
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
api_key = os.getenv("OPENAI_API_KEY")
SERPAPI_API_KEY = os.getenv("SERP_API_KEY")
os.environ["SERPAPI_API_KEY"] = SERPAPI_API_KEY

In [9]:
prompt = ChatPromptTemplate.from_template("给我讲一个关于的{topic}笑话")
model = ChatOpenAI(model = "gpt-4o-mini")
output_parser = StrOutputParser()

chain = prompt | model | output_parser
chain.invoke("冰淇淋")

'当然可以！这是一个冰淇淋的笑话：\n\n为什么冰淇淋总是很快乐？\n\n因为它有很多“甜蜜”的回忆！ \n\n希望这个笑话能让你笑一笑！'

In [13]:
prompt_value = prompt.invoke({"topic": "刺猬"})
prompt_value

ChatPromptValue(messages=[HumanMessage(content='给我讲一个关于的刺猬笑话', additional_kwargs={}, response_metadata={})])

In [15]:
prompt_value.to_messages()

[HumanMessage(content='给我讲一个关于的刺猬笑话', additional_kwargs={}, response_metadata={})]

In [27]:
from langchain_openai.llms import OpenAI

llm = OpenAI(model = "gpt-3.5-turbo-instruct")
llm.invoke(prompt_value)

'。\n\nAI: 为什么刺猬喜欢穿毛衣？因为它们喜欢把自己变成“毛刺猬”！'

In [37]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [43]:
vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding = OpenAIEmbeddings()
)

retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [45]:
chain.invoke("where did harrison work?")

'Harrison worked at Kensho.'

In [47]:
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

In [49]:
chain.invoke({"question": "where did harrison work", "language": "chinese"})

'Harrison 在 Kensho 工作。'